# Comparaisons entre différentes manières de prédiction de notes

Ce notebook présente la MSE et la MAE pour différentes façons de prédire les notes d'utilisateurs sur des séries.  
4000 utilisateurs pour 3000 séries.  
Au moins 4 notes pour chacun des utilisateurs.  
Les différents algorithmes comparés :
- kSVD
- NMF
- content


In [2]:
from utils.predictions_content import *
from utils.predictions_notes import *
from utils.collaborative import *

/Users/constancescherer/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [13]:
#path = "/Vrac/PLDAC_addic7ed/ratings/ratings_imdb/users"
path_ratings = "/Users/constancescherer/Desktop/ratings/ratings_imdb/users"
path_d_user = "/Users/constancescherer/Desktop/pickles/d_user.p"
#d_user= get_d_user(path)
with open(path_d_user, 'rb') as pickle_file:
    d_user = pickle.load(pickle_file)

liste_series = get_liste_series(d_user)
data = get_data(d_user)
all_data, num_user, num_item = get_all_data(data)
train, train_mat, test = get_train_test(num_user, num_item, all_data, test_size=20)
mean, u_means, i_means,U_ksvd, I_ksvd =  get_Uksvd_Iksvd(train, train_mat, num_user, num_item)
d_username_id, d_itemname_id, Full = create_sparse_mat(data)

path_series = "/Users/constancescherer/Desktop/addic7ed_final"
#path_series = '/Vrac/PLDAC_addic7ed/addic7ed_clean'
d_info, d_name = getDicts(path_series)
d_ind = reverse_dict(d_name)
d_titre_filename = get_d_titre_filename("titles/title-filename.txt")
d_filename_titre = reverse_dict(d_titre_filename)
d_id_username = reverse_dict(d_username_id)
d_id_serie = reverse_dict(d_itemname_id)

path_sim = "/Users/constancescherer/Desktop/pickles/sim.p"
# matrice des similarités cosinus
with open(path_sim, 'rb') as pickle_file:
    similarities = pickle.load(pickle_file)

In [6]:
def error_ksvd(train_mat, test) :
    ## Getting the truth values
    truth_tr = np.array([rating for (uid,iid),rating in train_mat.items()])
    truth_te = np.array([rating for uid,iid,rating in test])

    prediction_tr = np.array([pred_func_ksvd(u,i, U_ksvd, I_ksvd, u_means, i_means, mean) for (u,i),rating in train_mat.items()])
    prediction_te = np.array([pred_func_ksvd(u,i, U_ksvd, I_ksvd, u_means, i_means, mean) for u,i,rating in test])


    print("Training Error:")
    print("MSE:",  MSE_err(prediction_tr,truth_tr))
    print("MAE:",  MAE_err(prediction_tr,truth_tr))

    print("Test Error:")
    print("MSE:",  MSE_err(prediction_te,truth_te))
    print("MAE:",  MAE_err(prediction_te,truth_te))

In [20]:
def error_NMF(train_mat, test, num_user, num_item) :
    ## Getting the truth values
    truth_tr = np.array([rating for (uid,iid),rating in train_mat.items()])
    truth_te = np.array([rating for uid,iid,rating in test])
    
    prediction_tr, prediction_te = predictions_NMF(train_mat,test, 100, num_user, num_item)
    print("Training Error:")
    print("MSE:",  MSE_err(prediction_tr,truth_tr))
    print("MAE:",  MAE_err(prediction_tr,truth_tr))

    print("Test Error:")
    print("MSE:",  MSE_err(prediction_te,truth_te))
    print("MAE:",  MAE_err(prediction_te,truth_te))

In [17]:
def error_content(train_mat, test, d_name, d_user, d_ind, d_titre_filename, d_filename_titre, d_id_username, d_id_serie, similarities) :
    ## Getting the truth values
    truth_tr = np.array([rating for (uid,iid),rating in train_mat.items()])
    truth_te = np.array([rating for uid,iid,rating in test])

    prediction_tr = np.array([pred_content(u, i, d_name, d_user, d_ind, d_titre_filename, d_filename_titre, d_id_username, d_id_serie, similarities) for (u,i),rating in train_mat.items()])
    prediction_te = np.array([pred_content(u, i, d_name, d_user, d_ind, d_titre_filename, d_filename_titre, d_id_username, d_id_serie, similarities) for u,i,rating in test])


    print("Training Error:")
    print("MSE:",  MSE_err(prediction_tr,truth_tr))
    print("MAE:",  MAE_err(prediction_tr,truth_tr))

    print("Test Error:")
    print("MSE:",  MSE_err(prediction_te,truth_te))
    print("MAE:",  MAE_err(prediction_te,truth_te))

## kSVD

In [7]:
error_ksvd(train_mat, test)

Training Error:
MSE: 1.56699724060131
MAE: 0.7990219988643135
Test Error:
MSE: 5.645013042355757
MAE: 1.8067640310042217


## NMF

In [21]:
error_NMF(train_mat, test, num_user, num_item)

Training Error:
MSE: 0.001972234
MAE: 0.001972234
Test Error:
MSE: 9.03600293901543
MAE: 2.3188831741366642


## Content

In [19]:
error_content(train_mat, test, d_name, d_user, d_ind, d_titre_filename, d_filename_titre, d_id_username, d_id_serie, similarities)

Training Error:
MSE: 3.6809385084067014
MAE: 1.3689624502107582
Test Error:
MSE: 3.7667972895746593
MAE: 1.393093313739897


kSVD et content donnent de bons résultats en test. La NMF est un peu moins bonne en test mais elle ne prend pas en compte les biais des utilisateurs et des items (contrairement au kSVD).